In [1]:
import pandas as pd
from gensim import corpora, models
from gensim.similarities import MatrixSimilarity
from gensim.utils import SaveLoad
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re 
import pyLDAvis.gensim
import pickle
from collections import Counter
from gensim.matutils import corpus2csc, sparse2full, corpus2dense


In [2]:
SF = pd.read_csv('SF_jobs_new.csv')
SF.drop('Unnamed: 0',axis=1,inplace=True)
NYC = pd.read_csv('NYC_jobs_new.csv')
NYC.drop('Unnamed: 0',axis=1,inplace=True)
Seattle = pd.read_csv('Seattle_jobs_new.csv')
Seattle.drop('Unnamed: 0',axis=1,inplace=True)

In [3]:
def line_cleaner(line): # perform tokenization, select noun, Lemmertization etc on a line text
    rtext=[]    
    for w, tag in nltk.pos_tag(nltk.word_tokenize(line.lower())):  # Tokenization
        if tag in ['NN']:  # Keep only Nouns (for this special case)
            rtext.append(w)
    #text = re.sub("[^a-zA-Z+3#]"," ", line)
    
    #rtext = text.lower().split()
    
    wordnet_lemmatizer = WordNetLemmatizer()  # Lemmertization
    rtext = [wordnet_lemmatizer.lemmatize(w) for w in rtext]
            
    stop_words = set(stopwords.words("english") + ['e','g','etc','data','customer','experience','business','+','requirement']) # Filter out any stop words
    rtext = [w for w in rtext if not w in stop_words]
    
    return rtext # return list of tokens

def text_cleaner(html,Oneline=True): # return a single line of cleaned text

    soup_obj = BeautifulSoup(html,"lxml") # Get the html from the site

    for script in soup_obj(["script", "style"]):
        script.extract() # Remove these two elements from the BS4 object
    
    text = soup_obj.body.get_text('\n') # Get the text from this

    lines = (line.strip() for line in text.splitlines()) # break into lines
#    lines = [line for i, line in enumerate(lines) if i>16]
#    for line in lines:
#        print line
    chunks = (phrase.strip() for line in lines for phrase in line.split("  ")) # break multi-headlines into a line each
    
    def chunk_space(chunk):
        chunk_out = chunk + ' ' # Need to fix spacing issue
        return chunk_out  

    text = ''.join(chunk_space(chunk) for chunk in chunks if chunk).encode('utf-8') # Get rid of all blank lines and ends of line

    # Now clean out all of the unicode junk (this line works great!!!)
    try:
        text = text.decode('unicode_escape').encode('ascii', 'ignore') # Need this as some websites aren't formatted
    except:                                                            # in a way that this works, can occasionally throw
        return                                                         # an exception

    text = re.sub("[^a-zA-Z+3#]"," ", text)  # Now get rid of any terms that aren't words (include 3 for d3.js)
      
    text = Preprocessing_line(text)
    
    return text

In [25]:
cleanSentence = []
for i in range(SF.shape[0]):
    aa = BeautifulSoup(SF.job_description[i]).find_all('li')
    for sentence in aa:
        cleanSentence.append(line_cleaner(sentence.get_text()))

for i in range(NYC.shape[0]):
    bb = BeautifulSoup(NYC.job_description[i]).find_all('li')
    for sentence in bb:
        cleanSentence.append(line_cleaner(sentence.get_text()))

for i in range(Seattle.shape[0]):
    cc = BeautifulSoup(Seattle.job_description[i]).find_all('li')
    for sentence in bb:
        cleanSentence.append(line_cleaner(sentence.get_text()))

In [4]:
cleandoc = []
for i in range(SF.shape[0]):
    aa = BeautifulSoup(SF.job_description[i]).find_all('li')
    cleanSentence = []
    for sentence in aa:
        cleanSentence = cleanSentence + list(line_cleaner(sentence.get_text()))
    
    if len(cleanSentence):
        cleandoc.append(cleanSentence)
        
for i in range(NYC.shape[0]):
    aa = BeautifulSoup(NYC.job_description[i]).find_all('li')
    cleanSentence = []
    for sentence in aa:
        cleanSentence = cleanSentence + list(line_cleaner(sentence.get_text()))
    
    if len(cleanSentence):
        cleandoc.append(cleanSentence)
        
for i in range(Seattle.shape[0]):
    aa = BeautifulSoup(Seattle.job_description[i]).find_all('li')
    cleanSentence = []
    for sentence in aa:
        cleanSentence = cleanSentence + list(line_cleaner(sentence.get_text()))
    
    if len(cleanSentence):
        cleandoc.append(cleanSentence)

/Users/swang/anaconda/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /Users/swang/anaconda/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [61]:
cleanDescription = cleandoc

In [2]:
cleanDescription = []
for i in range(SF.shape[0]):
    cleanDescription.append(text_cleaner(SF.job_description[i]))
for i in range(NYC.shape[0]):
    cleanDescription.append(text_cleaner(NYC.job_description[i]))
for i in range(Seattle.shape[0]):
    cleanDescription.append(text_cleaner(Seattle.job_description[i]))
# save 
#with open('cleaned_sentence.txt','wb') as file1:
#   pickle.dump(cleanSentence,file1,protocol=2)

NameError: name 'cleanSentence' is not defined

In [12]:
cleanDescription = pd.read_pickle('cleaned_sentence.txt')

In [13]:
len(cleanDescription)

40046

In [65]:
def makeDict(myTweetList):
    """Create dictionary from list of tokenized documents"""
    return corpora.Dictionary(myTweetList)

def makeCorpus(myTweetList,myDict):
    """Create corpus from list of tokenized documents"""
    return [myDict.doc2bow(tweet) for tweet in myTweetList]

def createLDA(myCorpus, myDictionary,myTopics=10,myPasses=10,myIterations=50,myAlpha=0.001):
    """LDA model call function"""
    return models.LdaMulticore(myCorpus, id2word=myDictionary, num_topics=myTopics, passes=myPasses,
    iterations=myIterations,alpha=myAlpha)

In [66]:
kagDict   = makeDict(cleanDescription)
kagCorpus = makeCorpus(cleanDescription, kagDict)
kagLda = createLDA(kagCorpus, kagDict)

In [67]:
pyLDAvis.enable_notebook()
ldaViz = pyLDAvis.gensim.prepare(kagLda, kagCorpus, kagDict)
ldaViz

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
3      24.969681        1       1  0.028744 -0.039805
8      21.131297        1       2 -0.104403 -0.074041
0      16.301096        1       3  0.173499 -0.087527
9      14.024623        1       4 -0.023410 -0.047435
5       6.342225        1       5 -0.188143 -0.033260
1       4.589928        1       6  0.084234 -0.033536
6       4.008782        1       7 -0.038785  0.033053
2       3.718946        1       8  0.061506  0.130092
4       2.674550        1       9  0.036194  0.027811
7       2.238873        1      10 -0.029438  0.124648, topic_info=     Category        Freq            Term       Total  loglift  logprob
term                                                                   
1944  Default  298.000000            test  298.000000  30.0000  30.0000
862   Default  364.000000        security  364.000000  29.0000  29.0000
3923  Default  748.000000         ability  748.000000  28.0000  28.0000
335   Default  307.000000         network  307.000000  27.0000  27.0000
2963  Default  809.000000     development  809.000000  26.0000  26.0000
4070  Default  136.000000      automation  136.000000  25.0000  25.0000
2990  Default  344.000000        analysis  344.000000  24.0000  24.0000
3837  Default  695.000000       knowledge  695.000000  23.0000  23.0000
2373  Default  342.000000     information  342.000000  22.0000  22.0000
3709  Default  737.000000      management  737.000000  21.0000  21.0000
1129  Default  525.000000            team  525.000000  20.0000  20.0000
176   Default  584.000000            work  584.000000  19.0000  19.0000
1310  Default  295.000000         product  295.000000  18.0000  18.0000
2867  Default  214.000000            java  214.000000  17.0000  17.0000
1602  Default  294.000000     performance  294.000000  16.0000  16.0000
2233  Default  603.000000          design  603.000000  15.0000  15.0000
3713  Default  367.000000          system  367.000000  14.0000  14.0000
3707  Default  242.000000         quality  242.000000  13.0000  13.0000
3229  Default  265.000000     application  265.000000  12.0000  12.0000
3489  Default  302.000000         process  302.000000  11.0000  11.0000
4121  Default  212.000000             web  212.000000  10.0000  10.0000
4241  Default  445.000000        software  445.000000   9.0000   9.0000
450   Default  446.000000         project  446.000000   8.0000   8.0000
3270  Default  117.000000          oracle  117.000000   7.0000   7.0000
1092  Default  326.000000     environment  326.000000   6.0000   6.0000
947   Default  457.000000         support  457.000000   5.0000   5.0000
3274  Default  153.000000            code  153.000000   4.0000   4.0000
2033  Default  386.000000   communication  386.000000   3.0000   3.0000
3596  Default  124.000000          python  124.000000   2.0000   2.0000
3213  Default  232.000000     engineering  232.000000   1.0000   1.0000
...       ...         ...             ...         ...      ...      ...
2691  Topic10    8.402443            plan   74.740995   1.6137  -4.8318
4263  Topic10    4.247292            apps   20.092642   2.2451  -5.5141
1732  Topic10    8.414965          review   84.005019   1.4983  -4.8303
3713  Topic10   14.639379          system  367.564978   0.5760  -4.2766
3489  Topic10   12.683493         process  302.567280   0.6272  -4.4200
533   Topic10    6.243164     participate   62.432223   1.4966  -5.1289
2990  Topic10   12.556244        analysis  344.380948   0.4877  -4.4301
4274  Topic10    3.550385              hr   18.243494   2.1624  -5.6933
1129  Topic10   12.901152            team  525.683307   0.0918  -4.4030
2922  Topic10    2.855992           error   11.962334   2.3669  -5.9109
3787  Topic10    4.034843        accuracy   31.463625   1.7453  -5.5654
2977  Topic10    5.752381            user   88.103417   1.0703  -5.2107
1310  Topic10    8.793799         product  295.803633   0.2835  -4.78